In [27]:
# $ pip install "tensorflow>=1.7.0"
# $ pip install tensorflow-hub

import numpy as np
import pandas as pd
import re
import tensorflow as tf
import tensorflow_hub as tfhub
import sqlite3
from sqlite3 import Error

pd.set_option('display.max_colwidth', -1)





In [ ]:
wine_data = pd.read_csv("winemag-data-130k-v2.csv")

In [23]:
conn = sqlite3.connect('wine_data.sqlite')
c = conn.cursor()


In [25]:
wine_data.to_sql('wine_data', conn, if_exists = "replace")

wine_df = pd.read_sql('Select * from wine_data', conn)

c:\conda\envs\finance3\lib\site-packages\pandas\core\generic.py:2531: UserWarning: The spaces in these column names will not be changed. In pandas versions < 0.14, spaces were converted to underscores.
  dtype=dtype, method=method)


In [29]:
wine_df = pd.read_sql('Select * from wine_data', conn)

In [34]:
wine_df.head()
wine_df = wine_df.drop_duplicates('description')
wine_df = wine_df.dropna(subset=['price'])
wine_df

,index,Unnamed: 0,country,description,designation,points,price,province,region_1,region_2,taster_name,taster_twitter_handle,title,variety,winery
1,1,1,Portugal,"This is ripe and fruity, a wine that is smooth while still structured. Firm tannins are filled out with juicy red berry fruits and freshened with acidity. It's already drinkable, although it will certainly be better from 2016.",Avidagos,87,15.0,Douro,None,None,Roger Voss,@vossroger,Quinta dos Avidagos 2011 Avidagos Red (Douro),Portuguese Red,Quinta dos Avidagos
2,2,2,US,"Tart and snappy, the flavors of lime flesh and rind dominate. Some green pineapple pokes through, with crisp acidity underscoring the flavors. The wine was all stainless-steel fermented.",None,87,14.0,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,@paulgwine,Rainstorm 2013 Pinot Gris (Willamette Valley),Pinot Gris,Rainstorm
3,3,3,US,"Pineapple rind, lemon pith and orange blossom start off the aromas. The palate is a bit more opulent, with notes of honey-drizzled guava and mango giving way to a slightly astringent, semidry finish.",Reserve Late Harvest,87,13.0,Michigan,Lake Michigan Shore,None,Alexander Peartree,None,St. Julian 2013 Reserve Late Harvest Riesling (Lake Michigan Shore),Riesling,St. Julian
4,4,4,US,"Much like the regular bottling from 2012, this comes across as rather rough and tannic, with rustic, earthy, herbal characteristics. Nonetheless, if you think of it as a pleasantly unfussy country wine, it's a good companion to a hearty winter stew.",Vintner's Reserve Wild Child Block,87,65.0,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,@paulgwine,Sweet Cheeks 2012 Vintner's Reserve Wild Child Block Pinot Noir (Willamette Valley),Pinot Noir,Sweet Cheeks
5,5,5,Spain,"Blackberry and raspberry aromas show a typical Navarran whiff of green herbs and, in this case, horseradish. In the mouth, this is fairly full bodied, with tomatoey acidity. Spicy, herbal flavors complement dark plum fruit, while the finish is fresh but grabby.",Ars In Vitro,87,15.0,Northern Spain,Navarra,None,Michael Schachner,@wineschach,Tandem 2011 Ars In Vitro Tempranillo-Merlot (Navarra),Tempranillo-Merlot,Tandem
6,6,6,Italy,"Here's a bright, informal red that opens with aromas of candied berry, white pepper and savory herb that carry over to the palate. It's balanced with fresh acidity and soft tannins.",Belsito,87,16.0,Sicily & Sardinia,Vittoria,None,Kerin O’Keefe,@kerinokeefe,Terre di Giurfo 2013 Belsito Frappato (Vittoria),Frappato,Terre di Giurfo
7,7,7,France,"This dry and restrained wine offers spice in profusion. Balanced with acidity and a firm texture, it's very much for food.",None,87,24.0,Alsace,Alsace,None,Roger Voss,@vossroger,Trimbach 2012 Gewurztraminer (Alsace),Gewürztraminer,Trimbach
8,8,8,Germany,"Savory dried thyme notes accent sunnier flavors of preserved peach in this brisk, off-dry wine. It's fruity and fresh, with an elegant, sprightly footprint.",Shine,87,12.0,Rheinhessen,None,None,Anna Lee C. Iijima,None,Heinz Eifel 2013 Shine Gewürztraminer (Rheinhessen),Gewürztraminer,Heinz Eifel
9,9,9,France,This has great depth of flavor with its fresh apple and pear fruits and touch of spice. It's off dry while balanced with acidity and a crisp texture. Drink now.,Les Natures,87,27.0,Alsace,Alsace,None,Roger Voss,@vossroger,Jean-Baptiste Adam 2012 Les Natures Pinot Gris (Alsace),Pinot Gris,Jean-Baptiste Adam
10,10,10,US,"Soft, supple plum envelopes an oaky structure in this Cabernet, supported by 15% Merlot. Coffee and chocolate complete the picture, finishing strong at the end, resulting in a value-priced wine of attractive flavor and immediate accessibility.",Mountain Cuvée,87,19.0,California,Napa Valley,Napa,Virginie Boone,@vboone,Kirkland Signature 2011 Mountain Cuvée Cabernet Sauvignon (Napa Valley),Cabernet Sauvignon,Kirkland Signature


In [35]:
wine_df.to_sql('wine_data', conn, if_exists = "replace")

In [38]:
wine_df.shape

(111567, 15)

In [40]:
df = wine_df.groupby('variety').filter(lambda x: len(x) > 200)


,index,Unnamed: 0,country,description,designation,points,price,province,region_1,region_2,taster_name,taster_twitter_handle,title,variety,winery
1,1,1,Portugal,"This is ripe and fruity, a wine that is smooth while still structured. Firm tannins are filled out with juicy red berry fruits and freshened with acidity. It's already drinkable, although it will certainly be better from 2016.",Avidagos,87,15.0,Douro,None,None,Roger Voss,@vossroger,Quinta dos Avidagos 2011 Avidagos Red (Douro),Portuguese Red,Quinta dos Avidagos
2,2,2,US,"Tart and snappy, the flavors of lime flesh and rind dominate. Some green pineapple pokes through, with crisp acidity underscoring the flavors. The wine was all stainless-steel fermented.",None,87,14.0,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,@paulgwine,Rainstorm 2013 Pinot Gris (Willamette Valley),Pinot Gris,Rainstorm
3,3,3,US,"Pineapple rind, lemon pith and orange blossom start off the aromas. The palate is a bit more opulent, with notes of honey-drizzled guava and mango giving way to a slightly astringent, semidry finish.",Reserve Late Harvest,87,13.0,Michigan,Lake Michigan Shore,None,Alexander Peartree,None,St. Julian 2013 Reserve Late Harvest Riesling (Lake Michigan Shore),Riesling,St. Julian
4,4,4,US,"Much like the regular bottling from 2012, this comes across as rather rough and tannic, with rustic, earthy, herbal characteristics. Nonetheless, if you think of it as a pleasantly unfussy country wine, it's a good companion to a hearty winter stew.",Vintner's Reserve Wild Child Block,87,65.0,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,@paulgwine,Sweet Cheeks 2012 Vintner's Reserve Wild Child Block Pinot Noir (Willamette Valley),Pinot Noir,Sweet Cheeks
7,7,7,France,"This dry and restrained wine offers spice in profusion. Balanced with acidity and a firm texture, it's very much for food.",None,87,24.0,Alsace,Alsace,None,Roger Voss,@vossroger,Trimbach 2012 Gewurztraminer (Alsace),Gewürztraminer,Trimbach
8,8,8,Germany,"Savory dried thyme notes accent sunnier flavors of preserved peach in this brisk, off-dry wine. It's fruity and fresh, with an elegant, sprightly footprint.",Shine,87,12.0,Rheinhessen,None,None,Anna Lee C. Iijima,None,Heinz Eifel 2013 Shine Gewürztraminer (Rheinhessen),Gewürztraminer,Heinz Eifel
9,9,9,France,This has great depth of flavor with its fresh apple and pear fruits and touch of spice. It's off dry while balanced with acidity and a crisp texture. Drink now.,Les Natures,87,27.0,Alsace,Alsace,None,Roger Voss,@vossroger,Jean-Baptiste Adam 2012 Les Natures Pinot Gris (Alsace),Pinot Gris,Jean-Baptiste Adam
10,10,10,US,"Soft, supple plum envelopes an oaky structure in this Cabernet, supported by 15% Merlot. Coffee and chocolate complete the picture, finishing strong at the end, resulting in a value-priced wine of attractive flavor and immediate accessibility.",Mountain Cuvée,87,19.0,California,Napa Valley,Napa,Virginie Boone,@vboone,Kirkland Signature 2011 Mountain Cuvée Cabernet Sauvignon (Napa Valley),Cabernet Sauvignon,Kirkland Signature
11,11,11,France,"This is a dry wine, very spicy, with a tight, taut texture and strongly mineral character layered with citrus as well as pepper. It's a food wine with its almost crisp aftertaste.",None,87,30.0,Alsace,Alsace,None,Roger Voss,@vossroger,Leon Beyer 2012 Gewurztraminer (Alsace),Gewürztraminer,Leon Beyer
12,12,12,US,"Slightly reduced, this wine offers a chalky, tannic backbone to an otherwise juicy explosion of rich black cherry, the whole accented throughout by firm oak and cigar box.",None,87,34.0,California,Alexander Valley,Sonoma,Virginie Boone,@vboone,Louis M. Martini 2012 Cabernet Sauvignon (Alexander Valley),Cabernet Sauvignon,Louis M. Martini


In [42]:
df["color"] = ""
df.to_sql('wine_data', conn, if_exists = "replace")

c:\conda\envs\finance3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [49]:
c.execute('SELECT variety, count(variety) as vcount FROM wine_data group by variety ')
pd.read_sql('SELECT variety, count(variety) as vcount FROM wine_data group by variety', conn)
pd.read_sql('SELECT variety, count(variety) as vcount FROM wine_data group by variety', conn)

,variety,vcount
0,Aglianico,274
1,Albariño,432
2,Barbera,545
3,Bordeaux-style Red Blend,4958
4,Bordeaux-style White Blend,635
5,Cabernet Franc,1196
6,Cabernet Sauvignon,8760
7,Carmenère,525
8,Champagne Blend,1134
9,Chardonnay,10237


In [181]:
c.execute("update wine_data set color = 'red' where variety = 'Aglianico'  ")
conn.commit()

In [182]:
pd.read_sql("select color, count(color) from wine_data group by color" ,conn)

,color,count(color)
0,other,6173
1,red,59766
2,white,34322


In [189]:

df = pd.read_sql("select country, description,rating,price,province,title,variety,winery, color  from wine_data where color in ('red', 'white', 'other')", conn)
df.to_sql('wine_data', conn, if_exists = "replace")

In [190]:
df.head()
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100261 entries, 0 to 100260
Data columns (total 9 columns):
country        100228 non-null object
description    100261 non-null object
rating         100261 non-null int64
price          100261 non-null float64
province       100228 non-null object
title          100261 non-null object
variety        100261 non-null object
winery         100261 non-null object
color          100261 non-null object
dtypes: float64(1), int64(1), object(7)
memory usage: 6.9+ MB


In [191]:
df = df.dropna()
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 100228 entries, 0 to 100260
Data columns (total 9 columns):
country        100228 non-null object
description    100228 non-null object
rating         100228 non-null int64
price          100228 non-null float64
province       100228 non-null object
title          100228 non-null object
variety        100228 non-null object
winery         100228 non-null object
color          100228 non-null object
dtypes: float64(1), int64(1), object(7)
memory usage: 7.6+ MB


In [193]:
df.to_sql('wine_data', conn, if_exists = "replace")

wine_df = df.copy()
wine_df.head()


,country,description,rating,price,province,title,variety,winery,color
0,Portugal,"This is ripe and fruity, a wine that is smooth while still structured. Firm tannins are filled out with juicy red berry fruits and freshened with acidity. It's already drinkable, although it will certainly be better from 2016.",87,15.0,Douro,Quinta dos Avidagos 2011 Avidagos Red (Douro),Portuguese Red,Quinta dos Avidagos,red
1,US,"Tart and snappy, the flavors of lime flesh and rind dominate. Some green pineapple pokes through, with crisp acidity underscoring the flavors. The wine was all stainless-steel fermented.",87,14.0,Oregon,Rainstorm 2013 Pinot Gris (Willamette Valley),Pinot Gris,Rainstorm,white
2,US,"Pineapple rind, lemon pith and orange blossom start off the aromas. The palate is a bit more opulent, with notes of honey-drizzled guava and mango giving way to a slightly astringent, semidry finish.",87,13.0,Michigan,St. Julian 2013 Reserve Late Harvest Riesling (Lake Michigan Shore),Riesling,St. Julian,white
3,US,"Much like the regular bottling from 2012, this comes across as rather rough and tannic, with rustic, earthy, herbal characteristics. Nonetheless, if you think of it as a pleasantly unfussy country wine, it's a good companion to a hearty winter stew.",87,65.0,Oregon,Sweet Cheeks 2012 Vintner's Reserve Wild Child Block Pinot Noir (Willamette Valley),Pinot Noir,Sweet Cheeks,red
4,France,"This dry and restrained wine offers spice in profusion. Balanced with acidity and a firm texture, it's very much for food.",87,24.0,Alsace,Trimbach 2012 Gewurztraminer (Alsace),Gewürztraminer,Trimbach,white


In [39]:
embed = tfhub.Module("https://tfhub.dev/google/universal-sentence-encoder/2")

In [194]:
with tf.Session() as session:
  session.run([tf.global_variables_initializer(), tf.tables_initializer()])
  wine_embeddings = session.run(embed(list(wine_df.description)))
print(wine_embeddings.shape)

(100228, 512)


In [198]:
def recommend_engine(query, embedding_table = wine_embeddings):
    # Embed user query
    with tf.Session() as session:
        session.run([tf.global_variables_initializer(), tf.tables_initializer()])
        embedding = session.run(embed([query]))

    # Calculate similarity with all reviews
    similarity_score = np.dot(embedding, embedding_table.T)
    
    # Recommend
    recommendations = wine_df.copy()
    recommendations['recommendation'] = similarity_score.T
    recommendations = recommendations.sort_values('recommendation', ascending=False)
    recommendations = recommendations[['variety', 'title', 'price', 'description', 'recommendation']]

    return recommendations

In [200]:
query = "fruity, rich, easy to drink, sweet"

recommendation = recommend_engine(query)
print(query)

recommendation.head(5).T

fruity, rich, easy to drink, sweet


,28658,85049,83143,48466,87792
variety,Syrah,Cabernet Sauvignon,Cabernet Sauvignon,Cabernet Sauvignon,Cabernet Sauvignon
title,Dierberg 2008 Syrah (Santa Ynez Valley),Middle Sister NV Mischief Maker Cabernet Sauvignon (California),Kirkland Signature 2009 Cabernet Sauvignon (Alexander Valley),Wattle Creek 2010 Cabernet Sauvignon (Alexander Valley),Heron 2009 Cabernet Sauvignon (Mendocino)
price,34,12,9,42,13
description,"A nice Syrah, rich in berries, licorice, Dr. Pepper cola and spices, with soft, luxurious tannins. It's a little uncomplicated and sweet, but easy to drink now.","With an aroma like cherry cola, and fruity, almost sweet flavors, this medium-bodied wine is easy to drink.","Fruity in blackberry and cherry flavors, this is easy to drink, with a good acid-tannin balance.","Lots of flashy blackberry and cassis flavors in this Cabernet. Easy to drink, but feels way too soft to earn a higher score. Drink up.","A simple Cabernet, the kind to drink at a party and not fuss. It's dry and smooth, with berry and herb flavors."
recommendation,0.808436,0.805091,0.782239,0.780206,0.776853
